In [2]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import branca.colormap as cm
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import shape
from shapely import wkt
import numpy as np
import json

In [2]:
census_groups_df = './data/final/Census_Block_Groups_2010.geojson'

gdf_census = gpd.read_file(census_groups_df)
gdf_census.head()

,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,Shape__Area,Shape__Length,geometry
0,1,42,101,010800,1,421010108001,Block Group 1,G5030,S,161887,0,+39.9687580,-075.1997251,1.742508e+06,8200.327170,"POLYGON ((-75.19851 39.96945, -75.19744 39.969..."
1,2,42,101,010800,2,421010108002,Block Group 2,G5030,S,103778,0,+39.9665475,-075.2004455,1.117026e+06,4364.980144,"POLYGON ((-75.19783 39.96571, -75.20006 39.965..."
2,3,42,101,010900,2,421010109002,Block Group 2,G5030,S,43724,0,+39.9642929,-075.1896435,4.706347e+05,3048.109084,"POLYGON ((-75.18766 39.9645, -75.18755 39.9639..."
3,4,42,101,011000,2,421010110002,Block Group 2,G5030,S,108966,0,+39.9753585,-075.2113476,1.172871e+06,5169.004282,"POLYGON ((-75.20984 39.97351, -75.21221 39.973..."
4,5,42,101,011000,1,421010110001,Block Group 1,G5030,S,142244,0,+39.9724202,-075.2051689,1.531076e+06,10476.574129,"POLYGON ((-75.19855 39.9733, -75.19854 39.9730..."


In [3]:
census_isochrones_df = './data/isochrones.parquet'

df_iso = pd.read_parquet(census_isochrones_df)
df_iso.head()

,geometry,geoid,point_label,profile,time_limit,center_latitude,center_longitude,isochrone_area_square_meters
0,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",421010108001.0,center,foot,600,39.968766,-75.199727,1.545242e+06
1,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",421010108001.0,center,foot,1200,39.968766,-75.199727,7.552386e+06
2,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",421010108001.0,center,foot,1800,39.968766,-75.199727,1.737010e+07
3,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",421010108001.0,center,car,600,39.968766,-75.199727,1.988733e+08
4,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",421010108001.0,center,car,1200,39.968766,-75.199727,1.108233e+09


In [4]:
df_iso['geometry'] = df_iso['geometry'].apply(json.loads)
df_iso['geometry'] = df_iso['geometry'].apply(lambda x: shape(x['geometry']) if 'geometry' in x else shape(x))
gdf_iso = gpd.GeoDataFrame(df_iso, geometry='geometry')

In [5]:
gdf_iso = gdf_iso[gdf_iso['point_label'] == 'center']
gdf_iso['geoid'] = gdf_iso['geoid'].astype(float).round(0).astype(int).astype(str)
gdf_iso['time_limit'] = gdf_iso['time_limit'].astype(str)
gdf_iso['center_latitude'] = gdf_iso['center_latitude'].astype(str)
gdf_iso['center_longitude'] = gdf_iso['center_longitude'].astype(str)
gdf_iso['isochrone'] = gdf_iso[['geoid', 'time_limit', 'profile']].agg('-'.join, axis=1)
gdf_iso['center'] = gdf_iso[['center_latitude', 'center_longitude']].agg(','.join, axis=1)
gdf_iso = gdf_iso.drop(columns=['geoid', 'point_label', 'profile', 'time_limit', 'center_latitude', 'center_longitude'])
gdf_iso['area_m2'] = gdf_iso['isochrone_area_square_meters']
gdf_iso = gdf_iso[['isochrone', 'center', 'area_m2', 'geometry']]
gdf_iso = gdf_iso.set_crs(epsg=4326, allow_override=True)
gdf_iso.head()

,isochrone,center,area_m2,geometry
0,421010108001-600-foot,"39.96876598164982,-75.1997268790072",1.545242e+06,"POLYGON ((-75.20363 39.97252, -75.20363 39.972..."
1,421010108001-1200-foot,"39.96876598164982,-75.1997268790072",7.552386e+06,"POLYGON ((-75.19146 39.97075, -75.19335 39.972..."
2,421010108001-1800-foot,"39.96876598164982,-75.1997268790072",1.737010e+07,"POLYGON ((-75.19165 39.97022, -75.1917 39.9707..."
3,421010108001-600-car,"39.96876598164982,-75.1997268790072",1.988733e+08,"POLYGON ((-75.13683 39.95354, -75.13669 39.954..."
4,421010108001-1200-car,"39.96876598164982,-75.1997268790072",1.108233e+09,"POLYGON ((-75.17323 39.88167, -75.17309 39.881..."


Step 1: Creation of isochrone dataset:

In [37]:
gdf_census_copy = gdf_census.copy()
gdf_iso_merge   = gdf_iso.copy()

gdf_iso_merge[['lat','lon']] = (
    gdf_iso_merge['center']
      .str.split(',', expand=True)
      .astype(float)
)

gdf_iso_merge['geometry'] = gdf_iso_merge.apply(
    lambda r: Point(r['lon'], r['lat']),
    axis=1
)

gdf_iso_points = gpd.GeoDataFrame(
    gdf_iso_merge,
    geometry='geometry',
    crs="EPSG:4326"
).to_crs(gdf_census_copy.crs)

joined = gpd.sjoin(
    gdf_iso_points[['center','geometry']],
    gdf_census_copy[['geometry']],
    how='inner',
    predicate='within'
)

first_center = (
    joined
      .groupby('index_right', sort=False)
      .first()['center']
)

gdf_census_copy['first_iso_center'] = (
    gdf_census_copy.index
      .map(first_center)
      .astype(object)
      .fillna(None)
)

gdf_census_copy['first_iso_center'] = (
    gdf_census_copy['first_iso_center']
      .fillna("no_isochrone")
)

gdf_census_copy.head(30)

,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,Shape__Area,Shape__Length,geometry,first_iso_center
0,1,42,101,010800,1,421010108001,Block Group 1,G5030,S,161887,0,+39.9687580,-075.1997251,1.742508e+06,8200.327170,"POLYGON ((-75.19851 39.96945, -75.19744 39.969...","39.96876598164982,-75.1997268790072"
1,2,42,101,010800,2,421010108002,Block Group 2,G5030,S,103778,0,+39.9665475,-075.2004455,1.117026e+06,4364.980144,"POLYGON ((-75.19783 39.96571, -75.20006 39.965...","39.96655556342398,-75.2004472970602"
2,3,42,101,010900,2,421010109002,Block Group 2,G5030,S,43724,0,+39.9642929,-075.1896435,4.706347e+05,3048.109084,"POLYGON ((-75.18766 39.9645, -75.18755 39.9639...","39.96430097635668,-75.18964523537966"
3,4,42,101,011000,2,421010110002,Block Group 2,G5030,S,108966,0,+39.9753585,-075.2113476,1.172871e+06,5169.004282,"POLYGON ((-75.20984 39.97351, -75.21221 39.973...","39.9753665759928,-75.2113493640991"
4,5,42,101,011000,1,421010110001,Block Group 1,G5030,S,142244,0,+39.9724202,-075.2051689,1.531076e+06,10476.574129,"POLYGON ((-75.19855 39.9733, -75.19854 39.9730...","39.972428220268384,-75.20517069878659"
5,6,42,101,011000,3,421010110003,Block Group 3,G5030,S,111388,0,+39.9743352,-075.2073541,1.198951e+06,5524.835208,"POLYGON ((-75.20663 39.97526, -75.20436 39.974...","39.97434330350556,-75.20735593273575"
6,7,42,101,011100,4,421010111004,Block Group 4,G5030,S,147334,0,+39.9754452,-075.2281518,1.585865e+06,5762.100861,"POLYGON ((-75.22648 39.97485, -75.22647 39.974...","39.97545325216546,-75.22815363105919"
7,8,42,101,011100,2,421010111002,Block Group 2,G5030,S,249138,0,+39.9716381,-075.2209457,2.681653e+06,8140.953531,"POLYGON ((-75.21982 39.97364, -75.21948 39.973...","39.97164615142378,-75.2209475000264"
8,9,42,101,011100,3,421010111003,Block Group 3,G5030,S,86719,0,+39.9722444,-075.2259171,9.334006e+05,4519.861167,"POLYGON ((-75.22261 39.97213, -75.22367 39.971...","39.972252484959206,-75.22591884039616"
9,10,42,101,011100,1,421010111001,Block Group 1,G5030,S,622507,0,+39.9770521,-075.2189613,6.700488e+06,11047.938129,"POLYGON ((-75.21242 39.97399, -75.21221 39.973...","39.97706012947086,-75.21896321074453"


In [38]:
gdf_census_copy['INTPTLAT10'] = gdf_census_copy['INTPTLAT10'].str.replace('+', '').astype(float)
gdf_census_copy['INTPTLON10'] = gdf_census_copy['INTPTLON10'].astype(float)
gdf_census_copy = gdf_census_copy[['INTPTLAT10', 'INTPTLON10', 'first_iso_center', 'geometry']]
gdf_census_copy.head()

,INTPTLAT10,INTPTLON10,first_iso_center,geometry
0,39.968758,-75.199725,"39.96876598164982,-75.1997268790072","POLYGON ((-75.19851 39.96945, -75.19744 39.969..."
1,39.966547,-75.200446,"39.96655556342398,-75.2004472970602","POLYGON ((-75.19783 39.96571, -75.20006 39.965..."
2,39.964293,-75.189644,"39.96430097635668,-75.18964523537966","POLYGON ((-75.18766 39.9645, -75.18755 39.9639..."
3,39.975358,-75.211348,"39.9753665759928,-75.2113493640991","POLYGON ((-75.20984 39.97351, -75.21221 39.973..."
4,39.972420,-75.205169,"39.972428220268384,-75.20517069878659","POLYGON ((-75.19855 39.9733, -75.19854 39.9730..."


In [39]:
gdf_census_copy.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   INTPTLAT10        1336 non-null   float64 
 1   INTPTLON10        1336 non-null   float64 
 2   first_iso_center  1336 non-null   object  
 3   geometry          1336 non-null   geometry
dtypes: float64(2), geometry(1), object(1)
memory usage: 41.9+ KB


In [40]:
gdf_census_final = gdf_census_copy.copy()


In [41]:
gdf_census_final = gdf_census_final.dropna()
gdf_census_final.head()

,INTPTLAT10,INTPTLON10,first_iso_center,geometry
0,39.968758,-75.199725,"39.96876598164982,-75.1997268790072","POLYGON ((-75.19851 39.96945, -75.19744 39.969..."
1,39.966547,-75.200446,"39.96655556342398,-75.2004472970602","POLYGON ((-75.19783 39.96571, -75.20006 39.965..."
2,39.964293,-75.189644,"39.96430097635668,-75.18964523537966","POLYGON ((-75.18766 39.9645, -75.18755 39.9639..."
3,39.975358,-75.211348,"39.9753665759928,-75.2113493640991","POLYGON ((-75.20984 39.97351, -75.21221 39.973..."
4,39.972420,-75.205169,"39.972428220268384,-75.20517069878659","POLYGON ((-75.19855 39.9733, -75.19854 39.9730..."


In [42]:
features = []
for _, row in gdf_census_final.iterrows():
    if row['first_iso_center'] == "no_isochrone":
        fid = f"{row['INTPTLAT10']},{row['INTPTLON10']}"
    else:
        fid = row['first_iso_center']
    props = row.drop('geometry').to_dict()
    features.append({
        "type":       "Feature",
        "id":         fid,
        "properties": props,
        "geometry":   mapping(row.geometry)
    })

feature_collection = {
    "type":     "FeatureCollection",
    "features": features
}

with open('./final/census_blocks.geojson', 'w') as f:
    json.dump(feature_collection, f, indent=2)


Step 2: Creation of isochrone features dataset:

In [11]:
file_path = './data/yelp_academic_dataset_business.json'
df_yelp = pd.read_json(file_path, lines=True)

df_philly = df_yelp[df_yelp['city'] == 'Philadelphia'].copy()
df_philly.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14569 entries, 3 to 150336
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   14569 non-null  object 
 1   name          14569 non-null  object 
 2   address       14569 non-null  object 
 3   city          14569 non-null  object 
 4   state         14569 non-null  object 
 5   postal_code   14569 non-null  object 
 6   latitude      14569 non-null  float64
 7   longitude     14569 non-null  float64
 8   stars         14569 non-null  float64
 9   review_count  14569 non-null  int64  
 10  is_open       14569 non-null  int64  
 11  attributes    13399 non-null  object 
 12  categories    14560 non-null  object 
 13  hours         11785 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 1.7+ MB


In [12]:
geometry = [Point(xy) for xy in zip(df_philly['longitude'], df_philly['latitude'])]
gdf_business = gpd.GeoDataFrame(df_philly, geometry=geometry, crs='EPSG:4326')
if gdf_iso.crs != gdf_business.crs:
    gdf_iso = gdf_iso.to_crs(gdf_business.crs)
gdf_joined = gpd.sjoin(gdf_business, gdf_iso, how='inner', predicate='within')
avg_stars = gdf_joined.groupby('index_right')['stars'].mean()
gdf_iso['avg_stars'] = gdf_iso.index.map(avg_stars)
gdf_iso.head()

,isochrone,center,area_m2,geometry,avg_stars
0,421010108001-600-foot,"39.96876598164982,-75.1997268790072",1.545242e+06,"POLYGON ((-75.20363 39.97252, -75.20363 39.972...",3.708333
1,421010108001-1200-foot,"39.96876598164982,-75.1997268790072",7.552386e+06,"POLYGON ((-75.19146 39.97075, -75.19335 39.972...",3.519802
2,421010108001-1800-foot,"39.96876598164982,-75.1997268790072",1.737010e+07,"POLYGON ((-75.19165 39.97022, -75.1917 39.9707...",3.473648
3,421010108001-600-car,"39.96876598164982,-75.1997268790072",1.988733e+08,"POLYGON ((-75.13683 39.95354, -75.13669 39.954...",3.681063
4,421010108001-1200-car,"39.96876598164982,-75.1997268790072",1.108233e+09,"POLYGON ((-75.17323 39.88167, -75.17309 39.881...",3.642439


In [13]:
file_path_choice = './data/ChoiceNeighborhoods.geojson'

gdf_choice = gpd.read_file(file_path_choice)
gdf_choice.head()

,OBJECTID,NAME,TYPE,Shape__Area,Shape__Length,geometry
0,1,Mantua,Planning,1.120163e+07,13772.019457,"POLYGON ((-75.18717 39.96063, -75.18704 39.961..."
1,2,North Central,Implementation,1.575186e+07,17737.114794,"POLYGON ((-75.15644 39.98949, -75.15676 39.988..."
2,3,Sharswood-Blumberg,Planning,9.769110e+06,12755.506731,"POLYGON ((-75.17886 39.9814, -75.17918 39.98, ..."
3,4,Bartram,Planning,7.779424e+07,45980.410075,"POLYGON ((-75.20141 39.94977, -75.20311 39.949..."


In [14]:
if gdf_iso.crs != gdf_choice.crs:
    gdf_choice = gdf_choice.to_crs(gdf_iso.crs)

choice_union = gdf_choice.unary_union

gdf_iso['choice'] = gdf_iso.geometry.intersects(choice_union)
gdf_iso.head()

/var/folders/kz/3nj3xvg96sg9rnf641wm6p_h0000gn/T/ipykernel_75697/2804094208.py:4: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  choice_union = gdf_choice.unary_union


,isochrone,center,area_m2,geometry,avg_stars,choice
0,421010108001-600-foot,"39.96876598164982,-75.1997268790072",1.545242e+06,"POLYGON ((-75.20363 39.97252, -75.20363 39.972...",3.708333,True
1,421010108001-1200-foot,"39.96876598164982,-75.1997268790072",7.552386e+06,"POLYGON ((-75.19146 39.97075, -75.19335 39.972...",3.519802,True
2,421010108001-1800-foot,"39.96876598164982,-75.1997268790072",1.737010e+07,"POLYGON ((-75.19165 39.97022, -75.1917 39.9707...",3.473648,True
3,421010108001-600-car,"39.96876598164982,-75.1997268790072",1.988733e+08,"POLYGON ((-75.13683 39.95354, -75.13669 39.954...",3.681063,True
4,421010108001-1200-car,"39.96876598164982,-75.1997268790072",1.108233e+09,"POLYGON ((-75.17323 39.88167, -75.17309 39.881...",3.642439,True


In [15]:
file_path_no_thru_use= './data/No_thru_Trucks.geojson'

gdf_no_trucks = gpd.read_file(file_path_no_thru_use)
gdf_no_trucks.head()

,OBJECTID,ON_,FROM_,TO_,YEAR,SEG_ID,ST_CODE,ONEWAY,ST_NAME,STNAME,Shape__Length,geometry
0,1,POPLAR ST,COLLEGE AVE,POPLAR DR,1982,422113,65260,B,POPLAR,POPLAR ST,452.868792,"LINESTRING (-75.1854 39.97343, -75.18699 39.97..."
1,2,POPLAR ST,COLLEGE AVE,POPLAR DR,1982,421057,65260,B,POPLAR,POPLAR ST,154.904426,"LINESTRING (-75.18485 39.97336, -75.1854 39.97..."
2,3,POPLAR ST,COLLEGE AVE,POPLAR DR,1982,421059,65260,B,POPLAR,POPLAR ST,461.056776,"LINESTRING (-75.18215 39.97302, -75.18377 39.9..."
3,4,POPLAR ST,COLLEGE AVE,POPLAR DR,1982,421058,65260,B,POPLAR,POPLAR ST,305.879962,"LINESTRING (-75.18377 39.97323, -75.18485 39.9..."
4,5,BENJAMIN FRANKLIN PKWY,16TH ST,SPRING GARDEN,None,422216,16880,TF,BENJAMIN FRANKLIN,BENJAMIN FRANKLIN PKWY,962.380741,"LINESTRING (-75.17509 39.96119, -75.1768 39.96..."


In [17]:
gdf_no_trucks_proj = gdf_no_trucks.to_crs(epsg=4326)

joined = gpd.sjoin(gdf_no_trucks_proj, gdf_iso, how='inner', predicate='intersects')

length_by_block = joined.groupby('index_right')['Shape__Length'].sum()

gdf_iso['no_truck_length'] = gdf_iso.index.map(length_by_block).fillna(0)
gdf_iso.head()

,isochrone,center,area_m2,geometry,avg_stars,choice,no_truck_length
0,421010108001-600-foot,"39.96876598164982,-75.1997268790072",1.545242e+06,"POLYGON ((-75.20363 39.97252, -75.20363 39.972...",3.708333,True,1458.387879
1,421010108001-1200-foot,"39.96876598164982,-75.1997268790072",7.552386e+06,"POLYGON ((-75.19146 39.97075, -75.19335 39.972...",3.519802,True,4245.550548
2,421010108001-1800-foot,"39.96876598164982,-75.1997268790072",1.737010e+07,"POLYGON ((-75.19165 39.97022, -75.1917 39.9707...",3.473648,True,8506.445760
3,421010108001-600-car,"39.96876598164982,-75.1997268790072",1.988733e+08,"POLYGON ((-75.13683 39.95354, -75.13669 39.954...",3.681063,True,209322.298891
4,421010108001-1200-car,"39.96876598164982,-75.1997268790072",1.108233e+09,"POLYGON ((-75.17323 39.88167, -75.17309 39.881...",3.642439,True,835668.097031


In [18]:
file_path_pool= './data/PPR_Swimming_Pools.geojson'

gdf_swimming_pools = gpd.read_file(file_path_pool)
gdf_swimming_pools.head()

,OBJECTID,AMENITY_NAME,PARK_NAME,ADDRESS_911,ZIP_CODE,POOL_TYPE,POOL_STATUS,DATE_INSTALLED,COMMENTS,DATA_SOURCE,geometry
0,641,Fox Chase Pool,Fox Chase Recreation Center,7901 RIDGEWAY ST,19111,OUTDOOR,ACTIVE,NaT,None,Programs 2023; Nearmap 2023; Communications 2023,POINT (-75.08232 40.07367)
1,642,Mitchell Pool,Thomas Mitchell Playground,3600 WHITEHALL LN,19114,OUTDOOR,ACTIVE,2012-01-01 00:00:00+00:00,None,Programs 2023; Nearmap 2023,POINT (-74.99143 40.0683)
2,643,Jardel Pool,Thomas E Jardel Memorial Field,1400 COTTMAN AVE,19111,OUTDOOR,ACTIVE,NaT,None,Programs 2023; Nearmap 2023,POINT (-75.0759 40.05629)
3,644,Jacobs Pool,George Jacobs Playground,4500 LINDEN AVE,19136,OUTDOOR,ACTIVE,NaT,None,2024 September Site Visits,POINT (-75.00284 40.05266)
4,645,Morris Estates Pool,Morris Estate Cultural Center,1610 CHELTEN AVE,19141,OUTDOOR,ACTIVE,NaT,Emergency access is from Chelten around the bu...,Nearmap 2023,POINT (-75.14641 40.04992)


In [19]:
if gdf_iso.crs != gdf_swimming_pools.crs:
    gdf_swimming_pools = gdf_swimming_pools.to_crs(gdf_iso.crs)

gdf_iso['distance_pool'] = gdf_iso.geometry.apply(lambda poly: gdf_swimming_pools.distance(poly).min())
gdf_iso.head(10)

/var/folders/kz/3nj3xvg96sg9rnf641wm6p_h0000gn/T/ipykernel_75697/2713150581.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_iso['distance_pool'] = gdf_iso.geometry.apply(lambda poly: gdf_swimming_pools.distance(poly).min())


,isochrone,center,area_m2,geometry,avg_stars,choice,no_truck_length,distance_pool
0,421010108001-600-foot,"39.96876598164982,-75.1997268790072",1.545242e+06,"POLYGON ((-75.20363 39.97252, -75.20363 39.972...",3.708333,True,1458.387879,0.0
1,421010108001-1200-foot,"39.96876598164982,-75.1997268790072",7.552386e+06,"POLYGON ((-75.19146 39.97075, -75.19335 39.972...",3.519802,True,4245.550548,0.0
2,421010108001-1800-foot,"39.96876598164982,-75.1997268790072",1.737010e+07,"POLYGON ((-75.19165 39.97022, -75.1917 39.9707...",3.473648,True,8506.445760,0.0
3,421010108001-600-car,"39.96876598164982,-75.1997268790072",1.988733e+08,"POLYGON ((-75.13683 39.95354, -75.13669 39.954...",3.681063,True,209322.298891,0.0
4,421010108001-1200-car,"39.96876598164982,-75.1997268790072",1.108233e+09,"POLYGON ((-75.17323 39.88167, -75.17309 39.881...",3.642439,True,835668.097031,0.0
5,421010108001-1800-car,"39.96876598164982,-75.1997268790072",3.263262e+09,"POLYGON ((-75.31029 39.84624, -75.30249 39.846...",3.623086,True,944060.393496,0.0
6,421010108001-600-pt,"39.96876598164982,-75.1997268790072",1.728699e+06,"MULTIPOLYGON (((-75.20929 39.96784, -75.20932 ...",3.593750,True,729.032074,0.0
7,421010108001-1200-pt,"39.96876598164982,-75.1997268790072",8.499368e+06,"MULTIPOLYGON (((-75.21864 39.96727, -75.21864 ...",3.721763,True,14249.487275,0.0
8,421010108001-1800-pt,"39.96876598164982,-75.1997268790072",2.870245e+07,"MULTIPOLYGON (((-75.2418 39.97575, -75.2418 39...",3.623657,True,31231.511112,0.0
45,421010108002-600-foot,"39.96655556342398,-75.2004472970602",2.106224e+06,"POLYGON ((-75.20316 39.97205, -75.20336 39.972...",3.785714,True,729.355805,0.0


In [3]:
rtt_df = './data/rtt_data.csv'
df = pd.read_csv(rtt_df)
df.info()

/var/folders/kz/3nj3xvg96sg9rnf641wm6p_h0000gn/T/ipykernel_41046/922145774.py:2: DtypeWarning: Columns (26,34,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(rtt_df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296448 entries, 0 to 296447
Data columns (total 49 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   objectid                      296448 non-null  int64  
 1   document_id                   296448 non-null  int64  
 2   document_type                 296448 non-null  object 
 3   display_date                  296448 non-null  object 
 4   street_address                295592 non-null  object 
 5   zip_code                      289073 non-null  float64
 6   ward                          292480 non-null  float64
 7   grantors                      296371 non-null  object 
 8   grantees                      296408 non-null  object 
 9   cash_consideration            67159 non-null   float64
 10  other_consideration           40168 non-null   float64
 11  total_consideration           61288 non-null   float64
 12  assessed_value                37939 non-null

In [35]:
df['display_date'] = pd.to_datetime(df['display_date'], errors='coerce')

deed_df = df[
    (df['document_type'] == 'DEED') &
    (df['assessed_value'].notnull()) &
    (df['display_date'].dt.year > 2010)
]
deed_df.head(30)

,objectid,document_id,document_type,display_date,street_address,zip_code,ward,grantors,grantees,cash_consideration,...,street_postdir,reg_map_id,matched_regmap,opa_account_num,legal_remarks,discrepancy,property_count,record_id,lat,lng
146804,248538556,52308739,DEED,2011-01-17 05:00:00+00:00,4706-14 WESTMINSTER AVE,19131.0,6.0,BAYVIEW LOAN SERVICING LLC,4706 WESTMINSTER LLC,180000.0,...,NaN,059N240362,059N240405,NaN,NaN,False,1.0,523087399394031,39.968700,-75.216413
146805,248538557,52308740,DEED,2011-01-04 05:00:00+00:00,1648 W MENTOR ST,19141.0,13.0,FEDERAL HOME LOAN MORTGAGE CORPORATION; RITCHI...,KRAPIVIN ALEXANDER,27000.0,...,NaN,135N140083,135N140083,132396400.0,NaN,False,1.0,523087409394032,40.026083,-75.152427
146832,248539016,52309193,DEED,2011-01-17 05:00:00+00:00,2832 D ST,19134.0,7.0,HUNG PHAM TAN; NGUYEN HUNG V,PHAM HUNG TAN,1.0,...,NaN,035N040129,035N040129,71492500.0,NaN,False,1.0,523091939394540,39.992334,-75.122569
146859,248539469,52309631,DEED,2011-01-26 05:00:00+00:00,141 SIGEL ST,19148.0,1.0,DRINKHOUSE LORRAINE; DRINKHOUSE RICHARD,DRINKHOUSE LORRAINE; DRINKHOUSE RICHARD; DRINK...,1.0,...,NaN,014S050124,014S050124,11156800.0,NaN,False,1.0,523096319395044,39.924665,-75.148899
146860,248539470,52309632,DEED,2011-01-26 05:00:00+00:00,145 SIGEL ST,19148.0,1.0,DRINKHOUSE RICHARD,DRINKHOUSE LORRAINE; DRINKHOUSE RICHARD,1.0,...,NaN,014S050126,014S050126,11157000.0,NaN,False,1.0,523096329395045,39.924679,-75.149004
146914,248540360,52310499,DEED,2011-01-18 05:00:00+00:00,4834 PARRISH ST,19139.0,44.0,ROUSE LOUISE E ESTATE OF; ROUSE MARVIN ANTHONY,ROUSE LUCILLE; ROUSE MARVIN ANTHONY,1.0,...,NaN,060N050223,060N050223,441313800.0,NaN,False,1.0,523104999396035,39.967140,-75.218058
147208,248530614,52301249,DEED,2011-01-05 05:00:00+00:00,5245 N 6TH ST,19120.0,49.0,CASILLAS NARCISA ESTATE OF; TORRES MARIA,TORRES NANCY,1.0,...,NaN,123N160102,123N160102,492107600.0,NaN,False,1.0,523012499385233,40.031540,-75.133284
147236,248531061,52301673,DEED,2011-01-03 05:00:00+00:00,2854 N FRANKLIN ST,19133.0,37.0,FLORES MONICA; PINA MILAGROS C,CARTAGENA ALBERTO,65000.0,...,NaN,035N120212,035N120212,372073900.0,NaN,False,1.0,523016739385727,39.995404,-75.144062
147367,248533099,52303648,DEED,2011-01-11 05:00:00+00:00,1457 KERBAUGH ST,19140.0,13.0,COMMISSIONER OF RECORDS; JENKINS DAISY; JENKIN...,MORRIS MARVIN,1.0,...,NaN,101N120276,101N120276,132267400.0,NaN,False,1.0,523036489387986,40.013345,-75.152009
147368,248533100,52303649,DEED,2011-01-11 05:00:00+00:00,3815 PEARL ST,19104.0,24.0,COMMISSIONER OF RECORDS,WILSON ANTHONY; WILSON MICHAEL C,1.0,...,NaN,056N190126,056N190126,241159500.0,NaN,False,1.0,523036499387988,39.960181,-75.197659


In [37]:
gdf_deeds = gpd.GeoDataFrame(
    deed_df,
    geometry=gpd.points_from_xy(deed_df['lng'], deed_df['lat']),
    crs="EPSG:4326"
)
gdf_iso = gdf_iso.to_crs(epsg=4326)
gdf_joined = gpd.sjoin(
    gdf_deeds,
    gdf_iso,
    how='inner',
    predicate='within'
)
avg_prices = (
    gdf_joined
      .groupby('index_right')['assessed_value']
      .mean()
)
gdf_iso['avg_housing_price'] = gdf_iso.index.map(avg_prices)
gdf_iso[['avg_housing_price']].head()

,avg_housing_price
0,5719.0
1,5152.0
2,5584.0
3,8960.0
4,11168.0


In [38]:
foreclosure_df = df.loc[
    df['document_type'].isin(["SHERIFF'S DEED", "DEED SHERIFF"]) &
    df['assessed_value'].notnull() &
    (df['display_date'].dt.year > 2010)
]
foreclosure_df.head(30)

,objectid,document_id,document_type,display_date,street_address,zip_code,ward,grantors,grantees,cash_consideration,...,street_postdir,reg_map_id,matched_regmap,opa_account_num,legal_remarks,discrepancy,property_count,record_id,lat,lng
168705,248579236,52346716,DEED SHERIFF,2011-05-06 04:00:00+00:00,5313 PENTRIDGE ST,19143.0,51.0,MERCURY GROUP INC,SGNK LLC,15100.00,...,NaN,026S200245,026S200245,511127300.0,NaN,False,1.0,523467169439210,39.944541,-75.226786
169246,248587846,52353834,DEED SHERIFF,2011-05-09 04:00:00+00:00,5613 WARRINGTON AVE,19143.0,51.0,BERKMAN MICHAEL,OCONNOR CHRISTIAN; VAN ZANDT W SCOTT,32000.00,...,NaN,026S180364,026S180364,513304400.0,NaN,False,1.0,523538349448782,39.939836,-75.228823
169247,248587847,52353835,DEED SHERIFF,2011-05-09 04:00:00+00:00,1540 S 55TH ST,19143.0,51.0,BERKMAN MICHAEL,OCONNOR CHRISTIAN; VAN ZANDT W SCOTT,32000.00,...,NaN,028S150260,028S150260,514009300.0,NaN,False,1.0,523538359448783,39.937780,-75.223486
169248,248587848,52353836,DEED SHERIFF,2011-05-09 04:00:00+00:00,6014 ANGORA TER,19143.0,3.0,BERKMAN MICHAEL,OCONNOR CHRISTIAN; VAN ZANDT W SCOTT,3400.00,...,NaN,025S150130,025S150130,34030500.0,NaN,False,1.0,523538369448784,39.943351,-75.243149
169249,248587849,52353837,DEED SHERIFF,2011-05-24 04:00:00+00:00,2136 HAWORTH ST,19124.0,62.0,ALTIMAIRO ANN MARIE; BANDURA JOHN M,FEDERAL HOME LOAN MORTGAGE CORPORATION,6400.00,...,NaN,089N160198,089N160198,622024700.0,NaN,False,1.0,523538379448785,40.011219,-75.072308
169465,248576962,52344498,DEED SHERIFF,2011-04-26 04:00:00+00:00,7100 N 19TH ST,19126.0,10.0,TAGGART BOYD,BANK OF NEW YORK MELLON TR,45000.00,...,NaN,145N200296,145N200296,101121400.0,NaN,False,1.0,523444989436687,40.062425,-75.148113
169466,248576963,52344499,DEED SHERIFF,2011-04-20 04:00:00+00:00,1834-58 E WILLARD ST,19134.0,45.0,NEW PIKE CLEANERS INC,CRUZ CARMEN,10500.00,...,NaN,024N140222,NaN,885540340.0,NaN,False,1.0,523444999436688,39.996427,-75.110900
169617,248579249,52346719,DEED SHERIFF,2011-05-06 04:00:00+00:00,1436 W YORK ST,19132.0,16.0,TENNESSEE AVENUE LLC,PBB PROPERTY HOLDINGS LLC,16123.73,...,NaN,030N200315,030N200315,161273100.0,NaN,False,6.0,523467199439225,39.989376,-75.156982
170157,248587850,52353838,DEED SHERIFF,2011-05-24 04:00:00+00:00,6634 OAKLAND ST,19149.0,54.0,MCCUTCHEON MINGO III; MCGILL TONYA,US BANK NATIONAL ASSOCIATION TR,6900.00,...,NaN,139N100244,139N100244,541257800.0,LEGAL DESCRITPION DISCREPANCY FRONTAGE DIMENSI...,False,1.0,523538389448786,40.040067,-75.069411
170158,248587851,52353839,DEED SHERIFF,2011-05-24 04:00:00+00:00,7807 FAYETTE ST,19150.0,50.0,HARRIS KEITH B,EMC MORTGAGE CORPORATION,20000.00,...,NaN,143N240391,143N240391,501030800.0,NaN,False,1.0,523538399448788,40.071716,-75.163520


In [39]:
foreclosure_df = foreclosure_df.copy()
gdf_fore = gpd.GeoDataFrame(
    foreclosure_df,
    geometry=gpd.points_from_xy(foreclosure_df['lng'], foreclosure_df['lat']),
    crs="EPSG:4326"
)
gdf_joined = gpd.sjoin(
    gdf_fore,
    gdf_iso,
    how='inner',
    predicate='within'
)
counts = gdf_joined.groupby('index_right').size()
gdf_iso['foreclosure_count'] = (
    gdf_iso.index.map(counts)
              .fillna(0)
              .astype(int)
)
gdf_iso[['foreclosure_count']].head()

,foreclosure_count
0,13
1,53
2,80
3,1176
4,2706


In [43]:
gdf_iso['foreclosure_over_area'] = (gdf_iso['foreclosure_count'] / gdf_iso['area_m2'])

In [44]:
gdf_iso_features_final = gdf_iso.copy()
gdf_iso_features_final = gdf_iso_features_final.drop(columns=['geometry'])
gdf_iso_features_final.head()

,isochrone,center,area_m2,avg_stars,choice,no_truck_length,distance_pool,avg_housing_price,foreclosure_count,foreclosure_over_area
0,421010108001-600-foot,"39.96876598164982,-75.1997268790072",1.545242e+06,3.708333,True,1458.387879,0.0,5719.0,13,0.000008
1,421010108001-1200-foot,"39.96876598164982,-75.1997268790072",7.552386e+06,3.519802,True,4245.550548,0.0,5152.0,53,0.000007
2,421010108001-1800-foot,"39.96876598164982,-75.1997268790072",1.737010e+07,3.473648,True,8506.445760,0.0,5584.0,80,0.000005
3,421010108001-600-car,"39.96876598164982,-75.1997268790072",1.988733e+08,3.681063,True,209322.298891,0.0,8960.0,1176,0.000006
4,421010108001-1200-car,"39.96876598164982,-75.1997268790072",1.108233e+09,3.642439,True,835668.097031,0.0,11168.0,2706,0.000002


In [45]:
gdf_iso_features_final.to_csv("./final/iso_features.csv")

In [52]:
gdf_iso_final = gdf_iso.copy()

In [53]:
gdf_iso_final.drop(columns=['area_m2'])
gdf_iso_final.head()

,isochrone,center,area_m2,geometry
0,421010108001-600-foot,"39.96876598164982,-75.1997268790072",1.545242e+06,"POLYGON ((-75.20363 39.97252, -75.20363 39.972..."
1,421010108001-1200-foot,"39.96876598164982,-75.1997268790072",7.552386e+06,"POLYGON ((-75.19146 39.97075, -75.19335 39.972..."
2,421010108001-1800-foot,"39.96876598164982,-75.1997268790072",1.737010e+07,"POLYGON ((-75.19165 39.97022, -75.1917 39.9707..."
3,421010108001-600-car,"39.96876598164982,-75.1997268790072",1.988733e+08,"POLYGON ((-75.13683 39.95354, -75.13669 39.954..."
4,421010108001-1200-car,"39.96876598164982,-75.1997268790072",1.108233e+09,"POLYGON ((-75.17323 39.88167, -75.17309 39.881..."


In [54]:
gdf_iso_final['isochrone'] = gdf_iso_final['isochrone'].astype(str)
parts = gdf_iso_final['isochrone'].str.split('-', n=2, expand=True)
gdf_iso_final[['iso_id','distance_ft','mode']] = parts

coords = gdf_iso_final['center'].str.split(',', n=1, expand=True)
gdf_iso_final['lat'] = coords[0].astype(float)
gdf_iso_final['lon'] = coords[1].astype(float)

gdf_iso_final['lat_abs_str'] = gdf_iso_final['lat'].abs().astype(str)
decimals = gdf_iso_final['lat_abs_str'].str.split('.', n=1, expand=True)[1]
gdf_iso_final['lat_dec2'] = decimals.str[:2]
gdf_iso_final['lat_int']  = gdf_iso_final['lat'].astype(int).astype(str)
gdf_iso_final['lat_group'] = gdf_iso_final['lat_int'] + '_' + gdf_iso_final['lat_dec2']

for (dist, mode, lat_grp), sub in gdf_iso_final.groupby(
        ['distance_ft','mode','lat_group']
    ):
    fn = f"{dist}ft_{mode}_{lat_grp}.json"
    path = './final/tens' + fn
    to_drop = [
        'area_m2','iso_id','distance_ft','mode',
        'lat','lon','lat_abs_str','lat_dec2','lat_int','lat_group'
    ]
    sub.drop(columns=[c for c in to_drop if c in sub.columns]) \
       .to_file(path, driver='GeoJSON')
    print(f"Wrote {len(sub)} features:{fn}")


Wrote 3 features → 1200ft_car_39_88.json
Wrote 1 features → 1200ft_car_39_89.json
Wrote 8 features → 1200ft_car_39_90.json
Wrote 33 features → 1200ft_car_39_91.json
Wrote 62 features → 1200ft_car_39_92.json
Wrote 72 features → 1200ft_car_39_93.json
Wrote 88 features → 1200ft_car_39_94.json
Wrote 67 features → 1200ft_car_39_95.json
Wrote 74 features → 1200ft_car_39_96.json
Wrote 90 features → 1200ft_car_39_97.json
Wrote 83 features → 1200ft_car_39_98.json
Wrote 79 features → 1200ft_car_39_99.json
Wrote 50 features → 1200ft_car_40_00.json
Wrote 67 features → 1200ft_car_40_01.json
Wrote 104 features → 1200ft_car_40_02.json
Wrote 107 features → 1200ft_car_40_03.json
Wrote 108 features → 1200ft_car_40_04.json
Wrote 81 features → 1200ft_car_40_05.json
Wrote 61 features → 1200ft_car_40_06.json
Wrote 41 features → 1200ft_car_40_07.json
Wrote 20 features → 1200ft_car_40_08.json
Wrote 14 features → 1200ft_car_40_09.json
Wrote 10 features → 1200ft_car_40_10.json
Wrote 6 features → 1200ft_car_40_1